# 7장 새와 비행기 구별하기: 이미지 학습

고전 데이터셋인 CIFAR-10을 토치비전에서 다운로드할 수 있다

In [ ]:
pip install torchvision

Note: you may need to restart the kernel to use updated packages.


## 데이터셋 준비

In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import torch

torch.set_printoptions(edgeitems=2, linewidth=75)
torch.manual_seed(123)

c:\Users\danie\.conda\envs\ml\lib\site-packages\torch\random.py:42: UserWarning: Failed to initialize NumPy: module compiled against API version 0x10 but this version of numpy is 0xf (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:68.)
  return default_generator.manual_seed(seed)


In [ ]:
from torchvision import datasets
data_path = '../data-unversioned/p1ch7/'
cifar10 = datasets.CIFAR10(data_path, train=True, download=True) # <1>
cifar10_val = datasets.CIFAR10(data_path, train=False, download=True) # <2>

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
class_names = ['airplane','automobile','bird','cat','deer',
               'dog','frog','horse','ship','truck']

fig = plt.figure(figsize=(8,3))
num_classes = 10
for i in range(num_classes):
    ax = fig.add_subplot(2, 5, 1 + i, xticks=[], yticks=[])
    ax.set_title(class_names[i])
    img = next(img for img, label in cifar10 if label == i)
    plt.imshow(img)
plt.show()

CIFAR-10은 torch.utils.data.Dataset의 서브클래스이다.

In [ ]:
type(cifar10).__mro__

In [ ]:
len(cifar10)

CIFAR은 RGB로 이루어진 이미지이고, 다음은 자동차에 해당하는 정수값이 1인 PIL 형식의 이미지를 얻는 방법이다.

In [ ]:
img, label = cifar10[99]
img, label, class_names[label]

In [ ]:
plt.imshow(img)
plt.show()

이미지를 텐서로 변환하려면 torchvision의 transforms를 사용할 수 있다.

In [ ]:
from torchvision import transforms
dir(transforms)

이중에서 transforms.ToTensor를 사용하면 텐서로 만들어줌과 동시에 CxHxW 형식의 텐서로 변환해준다

In [ ]:
from torchvision import transforms

to_tensor = transforms.ToTensor()
img_t = to_tensor(img)
img_t.shape

변환 자체를 dataset.CIFAR10의 인자로 전달하는 것도 가능하다.

In [ ]:
tensor_cifar10 = datasets.CIFAR10(data_path, train=True, download=False,
                          transform=transforms.ToTensor())

In [ ]:
img_t, _ = tensor_cifar10[99]
type(img_t)

In [ ]:
img_t.shape, img_t.dtype

ToTensor() 변환으로 원래 0부터 255의 범위였던 이미지 값이 0.0부터 1.0 사이로 범위가 줄어든다

In [ ]:
img_t.min(), img_t.max()

In [ ]:
plt.imshow(img_t.permute(1, 2, 0))  # <1>
plt.show()

모든 채널이 평균값 0과 단위 표준편차를 가지기 위해 연산을 해줘야 하는데, 먼저 데이터셋이 반환하는 모든 텐서를 쌓아놓는다.

In [ ]:
imgs = torch.stack([img_t for img_t, _ in tensor_cifar10], dim=3)
imgs.shape

이제 view()함수를 통해 (3, 32, 32)였던 이미지를 (3, 1024)로 변환해준 후 평균과 표준편차를 구한다

In [ ]:
imgs.view(3, -1).mean(dim=1)  # <1>

In [ ]:
imgs.view(3, -1).std(dim=1)

이제 평균과 표준편차로 Normalize 변환을 초기화할 수 있다

In [ ]:
transforms.Normalize((0.4915, 0.4823, 0.4468), (0.2470, 0.2435, 0.2616))

In [ ]:
transformed_cifar10 = datasets.CIFAR10(
    data_path, train=True, download=False,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))

이제 Compose를 할 때 여러 변환과 동시에 정규화도 해줄 수 있다

In [ ]:
transformed_cifar10_val = datasets.CIFAR10(
    data_path, train=False, download=False,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))

In [ ]:
img_t, _ = transformed_cifar10[99]

plt.imshow(img_t.permute(1, 2, 0))
plt.show()

## 새와 비행기를 구별하기

먼저 데이터의 차원 정보를 맞춰야 한다, 굳이 Dataset의 서브클래스일 필요는 없으므로, __len__과 __getitem__이 정의되어 있으면 된다

In [ ]:
label_map = {0: 0, 2: 1}
class_names = ['airplane', 'bird']
cifar2 = [(img, label_map[label])
          for img, label in cifar10 
          if label in [0, 2]]
cifar2_val = [(img, label_map[label])
              for img, label in cifar10_val
              if label in [0, 2]]

이미지를 1차원 벡터로 간주하여 완전 연결 분류기를 만들수 있다. 피처는 3*32*32해서 3072개이다. 

In [ ]:
import torch.nn as nn

n_out = 2

model = nn.Sequential(
            nn.Linear(
                3072,  # <1>
                512,   # <2>
            ),
            nn.Tanh(),
            nn.Linear(
                512,   # <2>
                n_out, # <3>
            )
        )

이미지가 새인지, 비행기인지 분류하는 문제이므로 타겟 데이터를 원핫 벡터로 만들어줘야 한다. 또 한가지 새로운 사실은 각 인덱스에 대한 값을 확률로 표현할 수 있다는 점이다. 이렇게 모든 출력 요소의 합이 1이고 요소가 가질 수 있는 값이 0.0에서 1.0 사이로 만드는 함수가 소프트맥스 함수이다.

In [ ]:
def softmax(x):
    return torch.exp(x) / torch.exp(x).sum()

In [ ]:
x = torch.tensor([1.0, 2.0, 3.0])

softmax(x)

In [ ]:
softmax(x).sum()

In [ ]:
softmax = nn.Softmax(dim=1)

x = torch.tensor([[1.0, 2.0, 3.0],
                  [1.0, 2.0, 3.0]])

softmax(x)

이제 nn.Sequential로 모델을 완성해볼 수 있다. 이미지를 하나 골라서 확률을 구해보자

In [ ]:
model = nn.Sequential(
            nn.Linear(3072, 512),
            nn.Tanh(),
            nn.Linear(512, 2),
            nn.Softmax(dim=1))

In [ ]:
img, _ = cifar2[0]

plt.imshow(img.permute(1, 2, 0))
plt.show()

In [ ]:
img_batch = img.view(-1).unsqueeze(0)

In [ ]:
out = model(img_batch)
out

예측 벡터에서 뭐가 무슨 레이블인지 명시되어 있지 않다. 이를 위해 argmax로 값이 가장 큰 인덱스를 반환하는 함수를 쓸 수 있다. torch.max는 인자로 받은 차원에서 가장 값이 높은 인덱스를 반환한다.

In [ ]:
_, index = torch.max(out, dim=1)

index

In [ ]:
out = torch.tensor([
    [0.6, 0.4],
    [0.9, 0.1],
    [0.3, 0.7],
    [0.2, 0.8],
])
class_index = torch.tensor([0, 0, 1, 1]).unsqueeze(1)

truth = torch.zeros((4,2))
truth.scatter_(dim=1, index=class_index, value=1.0)
truth

In [ ]:
def mse(out):
    return ((out - truth) ** 2).sum(dim=1).mean()
mse(out)

In [ ]:
out.gather(dim=1, index=class_index)

In [ ]:
def likelihood(out):
    prod = 1.0
    for x in out.gather(dim=1, index=class_index):
        prod *= x
    return prod

likelihood(out)

In [ ]:
def neg_log_likelihood(out):
    return -likelihood(out).log()

neg_log_likelihood(out)

In [ ]:
out0 = out.clone().detach()
out0[0] = torch.tensor([0.9, 0.1]) # more right

out2 = out.clone().detach()
out2[0] = torch.tensor([0.4, 0.6]) # slightly wrong

out3 = out.clone().detach()
out3[0] = torch.tensor([0.1, 0.9]) # very wrong

mse_comparison = torch.tensor([mse(o) for o in [out0, out, out2, out3]])
mse_comparison

In [ ]:
((mse_comparison / mse_comparison[1]) - 1) * 100

In [ ]:
nll_comparison = torch.tensor([neg_log_likelihood(o) 
                               for o in [out0, out, out2, out3]])
nll_comparison

In [ ]:
((nll_comparison / nll_comparison[1]) - 1) * 100

In [ ]:
softmax = nn.Softmax(dim=1)

log_softmax = nn.LogSoftmax(dim=1)

x = torch.tensor([[0.0, 104.0]])

softmax(x)

In [ ]:
softmax = nn.Softmax(dim=1)

log_softmax = nn.LogSoftmax(dim=1)

x = torch.tensor([[0.0, 104.0]])

softmax(x)

In [ ]:
torch.log(softmax(x))

In [ ]:
log_softmax(x)

In [ ]:
torch.exp(log_softmax(x))

이런 분류 문제에서는 단순히 손실값을 줄이는 것이 목표가 아니라, 가능도가 높고 다른 타겟 인덱스에 대한 확률이 낮아야 한다. 이러한 지표를 가르키는 함수 중에는 nn.NLLLoss가 있다. 이 손실 함수는 입력을 확률 대신 로그 확률의 텐서를 받는다. 이를 위해 마지막에 nn.LogSoftmax 함수로 반환해주자.

In [ ]:
model = nn.Sequential(
            nn.Linear(3072, 512),
            nn.Tanh(),
            nn.Linear(512, 2),
            nn.LogSoftmax(dim=1))

In [ ]:
loss = nn.NLLLoss()

In [ ]:
img, label = cifar2[0]

out = model(img.view(-1).unsqueeze(0))

loss(out, torch.tensor([label]))

이제 본격적으로 훈련해 볼 수 있다. 단일 배치에 1만개의 이미지를 모두 평가하는 것은 너무 많으므로 내부 루프 안에 한번의 하나의 샘플을 평가하고 단일 샘플에 대해 역전파해 볼 수 있다. 

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

model = nn.Sequential(
            nn.Linear(3072, 512),
            nn.Tanh(),
            nn.Linear(512, 2),
            nn.LogSoftmax(dim=1))

learning_rate = 1e-2

optimizer = optim.SGD(model.parameters(), lr=learning_rate)

loss_fn = nn.NLLLoss()

n_epochs = 100

for epoch in range(n_epochs):
    for img, label in cifar2:
        out = model(img.view(-1).unsqueeze(0))
        loss = loss_fn(out, torch.tensor([label]))
                
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print("Epoch: %d, Loss: %f" % (epoch, float(loss)))

우리는 한번에 하나의 아이템을 골라 크기가 1인 미니 배치를 만드는데, DataLoader 클래스를 통해 배치 사이즈와 랜덤 샘플링을 할 shuffle=True를 사용하여 인덱스를 섞을 수 있다.

In [ ]:
train_loader = torch.utils.data.DataLoader(cifar2, batch_size=64,
                                           shuffle=True)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

train_loader = torch.utils.data.DataLoader(cifar2, batch_size=64,
                                           shuffle=True)

model = nn.Sequential(
            nn.Linear(3072, 128),
            nn.Tanh(),
            nn.Linear(128, 2),
            nn.LogSoftmax(dim=1))

learning_rate = 1e-2

optimizer = optim.SGD(model.parameters(), lr=learning_rate)

loss_fn = nn.NLLLoss()

n_epochs = 100

for epoch in range(n_epochs):
    for imgs, labels in train_loader:
        outputs = model(imgs.view(imgs.shape[0], -1))
        loss = loss_fn(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print("Epoch: %d, Loss: %f" % (epoch, float(loss)))

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

train_loader = torch.utils.data.DataLoader(cifar2, batch_size=64,
                                           shuffle=True)

model = nn.Sequential(
            nn.Linear(3072, 512),
            nn.Tanh(),
            nn.Linear(512, 2),
            nn.LogSoftmax(dim=1))

learning_rate = 1e-2

optimizer = optim.SGD(model.parameters(), lr=learning_rate)

loss_fn = nn.NLLLoss()

n_epochs = 100

for epoch in range(n_epochs):
    for imgs, labels in train_loader:
        outputs = model(imgs.view(imgs.shape[0], -1))
        loss = loss_fn(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print("Epoch: %d, Loss: %f" % (epoch, float(loss)))

In [ ]:
train_loader = torch.utils.data.DataLoader(cifar2, batch_size=64,
                                           shuffle=False)

correct = 0
total = 0

with torch.no_grad():
    for imgs, labels in train_loader:
        outputs = model(imgs.view(imgs.shape[0], -1))
        _, predicted = torch.max(outputs, dim=1)
        total += labels.shape[0]
        correct += int((predicted == labels).sum())
        
print("Accuracy: %f" % (correct / total))

In [ ]:
val_loader = torch.utils.data.DataLoader(cifar2_val, batch_size=64,
                                         shuffle=False)

correct = 0
total = 0

with torch.no_grad():
    for imgs, labels in val_loader:
        outputs = model(imgs.view(imgs.shape[0], -1))
        _, predicted = torch.max(outputs, dim=1)
        total += labels.shape[0]
        correct += int((predicted == labels).sum())
        
print("Accuracy: %f" % (correct / total))

In [ ]:
model = nn.Sequential(
            nn.Linear(3072, 1024),
            nn.Tanh(),
            nn.Linear(1024, 512),
            nn.Tanh(),
            nn.Linear(512, 128),
            nn.Tanh(),
            nn.Linear(128, 2),
            nn.LogSoftmax(dim=1))

사실 nn.LogSoftmax와 nn.NLLLoss는 CrossEntropyLoss()와 같다. 대부분의 경우 Softmax를 하지 않고 마지막에 nn.CrossEntropyLoss를 사용할 수 있다

In [ ]:
model = nn.Sequential(
            nn.Linear(3072, 1024),
            nn.Tanh(),
            nn.Linear(1024, 512),
            nn.Tanh(),
            nn.Linear(512, 128),
            nn.Tanh(),
            nn.Linear(128, 2))

loss_fn = nn.CrossEntropyLoss()

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

train_loader = torch.utils.data.DataLoader(cifar2, batch_size=64,
                                           shuffle=True)

model = nn.Sequential(
            nn.Linear(3072, 1024),
            nn.Tanh(),
            nn.Linear(1024, 512),
            nn.Tanh(),
            nn.Linear(512, 128),
            nn.Tanh(),
            nn.Linear(128, 2))

learning_rate = 1e-2

optimizer = optim.SGD(model.parameters(), lr=learning_rate)

loss_fn = nn.CrossEntropyLoss()

n_epochs = 100

for epoch in range(n_epochs):
    for imgs, labels in train_loader:
        outputs = model(imgs.view(imgs.shape[0], -1))
        loss = loss_fn(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print("Epoch: %d, Loss: %f" % (epoch, float(loss)))

## 연습문제
1. torchvision을 사용하여 데이터를 임의로 잘라내보자\
a.원래의 이미지와 비교하여 어떤 점이 다른가?\
b. 동일한 이미지를 다시 처리하면 어떻게 되는가?\
c. 랜덤하게 크롭된 이미지로 훈련시킨 결과는 어떠한가?

In [4]:
import torchvision
from torchvision import datasets
from torchvision import transforms
data_path = '../data-unversioned/p1ch7/'
transformed_cifar10 = datasets.CIFAR10(
    data_path, train=True, download=False,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))

In [ ]:
import matplotlib.pyplot as plt

In [8]:
transformed_cifar10.shape

AttributeError: 'CIFAR10' object has no attribute 'shape'

In [3]:
from torchvision import transforms
img, _ = cifar10[99]
n_img = transforms.functional.crop(img, 50, 50, 50, 50)
n_img

NameError: name 'cifar10' is not defined